In [24]:
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import TweetTokenizer
from urllib import request
from gensim.models import Word2Vec
import numpy as np
import gensim
import string
import nltk
import os, ssl
import re

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/kswierkot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kswierkot/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kswierkot/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kswierkot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
url_dem_platform_2016 = 'https://www.presidency.ucsb.edu/documents/2016-democratic-party-platform'
url_debate_las_vegas_2016 = 'https://www.presidency.ucsb.edu/documents/presidential-debate-the-university-nevada-las-vegas'
url_debate_st_louis_2016 = 'https://www.presidency.ucsb.edu/documents/presidential-debate-washington-university-st-louis-missouri'
url_rep_platform_2016 = 'https://www.presidency.ucsb.edu/documents/2016-republican-party-platform'
url_dem_platform_1980 = 'https://www.presidency.ucsb.edu/documents/1980-democratic-party-platform'
url_rep_platform_1980 = 'https://www.presidency.ucsb.edu/documents/republican-party-platform-1980'
url_debate_cleveland_1980 = 'https://www.presidency.ucsb.edu/documents/presidential-debate-cleveland'
url_debate_baltimore_1980 = 'https://www.presidency.ucsb.edu/documents/presidential-debate-baltimore-reagan-anderson'
en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop = en_stop.union(set(['republican', 'democratic']))
tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

In [32]:
from nltk.tokenize import sent_tokenize, word_tokenize
html_dem_platform_2016 = request.urlopen(url_dem_platform_2016).read().decode('utf8')
raw_dem_platform_2016 = BeautifulSoup(html_dem_platform_2016, 'html.parser')
raw_dem_platform_2016 = [p.get_text() for p in raw_dem_platform_2016.select('div.field-docs-content > p')]
raw_dem_platform_2016 = ' '.join(raw_dem_platform_2016)
raw_dem_platform_2016 = re.sub(r'\d+', '', raw_dem_platform_2016)

html_rep_platform_2016 = request.urlopen(url_rep_platform_2016).read().decode('utf8')
raw_rep_platform_2016 = BeautifulSoup(html_rep_platform_2016, 'html.parser')
raw_rep_platform_2016 = [p.get_text() for p in raw_rep_platform_2016.select('div.field-docs-content > p')]
raw_rep_platform_2016 = ' '.join(raw_rep_platform_2016)
raw_rep_platform_2016 = re.sub(r'\d+', '', raw_rep_platform_2016)

data_dem_platform_2016 = []
for i in sent_tokenize(raw_dem_platform_2016):
    temp = []
    for j in word_tokenize(i):
        temp.append(j.lower())
        
    data_dem_platform_2016.append(temp)

    
data_rep_platform_2016 = []
for i in sent_tokenize(raw_rep_platform_2016):
    temp = []
    for j in word_tokenize(i):
        temp.append(j.lower())
        
    data_rep_platform_2016.append(temp)

In [49]:
# Create CBOW model
model_dem_cbow = gensim.models.Word2Vec(data_dem_platform_2016, min_count = 1,
                              vector_size = 50, window = 1)
 
# Create Skip Gram model
model_dem_gram = gensim.models.Word2Vec(data_dem_platform_2016, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)
 

In [19]:
model_dem_cbow.wv

In [29]:
import numpy as np

In [38]:
# GRADED FUNCTION: cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u, v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u**2))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / np.dot(norm_u, norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

In [51]:
cos_similarity = cosine_similarity(model_dem_cbow.wv['democrats'], model_dem_cbow.wv['also'])
cos_similarity

0.9908463

In [43]:
# GRADED FUNCTION: complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    ### START CODE HERE ###
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    ### END CODE HERE ###
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c] :
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity((e_b - e_a), (word_to_vec_map[w] - e_c))
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        ### END CODE HERE ###
        
    return best_word

In [ ]:
test = complete_analogy(*triad,word_to_vec_map)